In [1]:
import os
import sys
sys.path.append('src/')

import cv2
import time
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from IPython.display import clear_output

import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from dataset import ClassifierDataset, TARGETS

In [2]:
df = pd.read_csv("data/train_unique.csv", sep=",")
print(df.shape)
df.head()

(20345, 6)


,object_id,name,description,group,img_name,fold
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg,0
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg,1
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg,2
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg,0
4,6339754,Сабля.,NaN,Оружие,2592073.jpg,0


In [3]:
df = pd.read_csv("data/train.csv", sep=";")
print(df.shape)
df.head()

(20345, 5)


,object_id,name,description,group,img_name
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg
4,6339754,Сабля.,NaN,Оружие,2592073.jpg


In [4]:
sub = pd.read_csv("data/submission.csv", sep=";")
print(sub.shape)
sub.head()

(20345, 3)


,object_id,img_name,group
0,10669820,7862029.jpg,Техника
1,4489444,9461061.jpg,Техника
2,8722586,5095122.jpg,Техника
3,3712248,551422.jpg,Техника
4,6339754,2592073.jpg,Техника


In [5]:
ROOT = "data"

@dataclass
class CFG:
    # dataset
    path_to_csv_file: str = f"{ROOT}/train.csv"
    mode: str = "train"
    fold: int = 3
    num_workes: int = 6
    group: str = "Egor"
    description: str = '''resnet-34'''

    # training
    num_classes: int = 15
    batch_size: int = 60
    wandb_project: str = 'CP-classification-baseline'
    default_root_dir: str = 'weights'
    checkpoints_dir: str = 'weights/checkpoints'
    lr: float = 3e-4
    weight_decay: float = 1e-1
    max_epochs: int = 50

In [6]:
CFG = CFG()

In [7]:
test_dataset = ClassifierDataset(
    mode="test",
    data_path="test/subm_test.csv",
    fold = 21,
)

In [38]:
train_dataset = ClassifierDataset(
    mode="train",
    data_path=CFG.path_to_csv_file,
    fold = CFG.fold,
)

val_dataset = ClassifierDataset(
    mode="eval",
    data_path=CFG.path_to_csv_file,
    fold=CFG.fold
)

/home/jovyan/.local/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jovyan/.local/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [8]:
TARGETS

['Археология',
 'Оружие',
 'Прочие',
 'Нумизматика',
 'Фото, негативы',
 'Редкие книги',
 'Документы',
 'Печатная продукция',
 'ДПИ',
 'Скульптура',
 'Графика',
 'Техника',
 'Живопись',
 'Естественнонауч.коллекция',
 'Минералогия']

In [40]:
df = pd.read_csv("data/train_unique.csv")

targets = df[df.fold != CFG.fold]["group"].apply(lambda x: TARGETS.index(x)).values
all_targets = pd.read_csv("data/train_unique.csv")["group"].apply(lambda x: TARGETS.index(x)).values

class_counts = np.bincount(all_targets)
class_weights = 1. / class_counts
weights = class_weights[targets]

sampler = WeightedRandomSampler(weights, len(weights))

In [9]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=47)

In [41]:
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, sampler=sampler, shuffle=False, num_workers=CFG.num_workes)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workes)

In [10]:
%%time
for batch_idx, batch in enumerate(test_loader):
    print(f"Batch {batch_idx}:")
    print(f"Keys {batch.keys()}")
    print(f"Images shape {batch['image'].shape}")
    print(batch['label'][:5])
    break

Batch 0:
Keys dict_keys(['image', 'label', 'object_id', 'img_name'])
Images shape torch.Size([1, 3, 512, 512])
tensor([-1])
CPU times: user 78.2 ms, sys: 812 ms, total: 890 ms
Wall time: 2.36 s


In [30]:
image = test_dataset[0]["image"]

plt.imshow(image.permute(1,2,0))
plt.show()

time.sleep(3)
clear_output()

In [14]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

class Classifier(pl.LightningModule):

    def __init__(self, config, pretrain=None, **kwargs):
        super().__init__()
        self.args = config
        self.learning_rate = config.lr
        self.save_hyperparameters()    

        self.model = torchvision.models.resnet34(torchvision.models.ResNet34_Weights.DEFAULT)
        self.model.fc = torch.nn.Linear(512, len(TARGETS))
    
        if pretrain is not None:
            self.model.load_state_dict(torch.load(pretrain), strict=False)
            print(f'resumed from {pretrain}')

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, batch):
        return self.model(batch['image'])

    def compute_f1(self, logits, labels):
        preds = torch.argmax(logits, dim=-1)
        return f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
    
    def training_step(self, batch, batch_idx):
        logits = self.model(batch['image'])

        labels = batch['label']

        loss = self.loss_fn(logits, labels)
        f1 = self.compute_f1(logits, labels)

        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_f1", f1, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            logits = self.model(batch['image'])
        
        labels = batch['label']

        loss = self.loss_fn(logits, labels)
        f1 = self.compute_f1(logits, labels)
    
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_f1", f1, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=self.args.weight_decay, eps=1e-5)
        lr_scheduler = {
                        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                                                'min',
                                                                                factor=0.5,
                                                                                patience=3,
                                                                                threshold=0.07,),
                        'interval': 'epoch',
                        'frequency': 1,
                        'monitor': "train_loss",
                        'name': 'lr/reduce_on_plateau'
                        }
        return [optimizer], [lr_scheduler]

In [15]:
# wandb_name = f"{CFG.group}_{CFG.description}_fold{CFG.fold}_"

# wandb.login()
# #
# wandb_logger = WandbLogger(project = CFG.wandb_project, name=wandb_name, group=CFG.group)

# wandb.init()
# wandb.config.update({k: v for k, v in CFG.__dict__.items() if not k.startswith("__")})

In [16]:
# model = Classifier(config=CFG, pretrain=None)
# # model = Classifier.load_from_checkpoint('weights/eff_v2_m/Egor_baseline_fold4__epochepoch=9-val_loss=0.586.ckpt', config=CFG)

# lr_monitor = LearningRateMonitor(logging_interval='epoch')

# checkpoint_callback = ModelCheckpoint(
#     dirpath=CFG.checkpoints_dir,
#     monitor='val_loss',
#     save_top_k=3,
#     filename=wandb_name + '_epoch{epoch}-{val_loss:.3f}',
#     mode='min',
#     save_weights_only=True,
# )

In [17]:
# trainer = pl.Trainer(logger=wandb_logger,
#                      default_root_dir=CFG.default_root_dir,
#                      accelerator='cuda',
#                      devices=[0],
#                      callbacks=[checkpoint_callback, lr_monitor],
#                      max_epochs=100,
#                      check_val_every_n_epoch=1,
#                      precision=32,
#                      )

# trainer.fit(model, train_dataloaders = train_loader, val_dataloaders = val_loader)

In [42]:
class Ensemble():
    def __init__(self,
                 path_to_models: str,
                 device: str = "cuda"):
        self.device = device

        models = []
        for path in os.listdir(path_to_models):
            if ".ckpt" in path:
                path = os.path.join(path_to_models, path)
                print(path)
                model = Classifier.load_from_checkpoint(path, config=CFG).to(device).eval()
                models.append(model)

        self.models = models
    
    def predict(self, batch):
        batch["image"] = batch["image"].to(self.device)
        preds = np.zeros(15)
        for model in self.models:
            with torch.no_grad():
                logits = model.forward(batch).cpu().detach().float()
            pred = F.softmax(logits, dim=1)[0].numpy()
            preds += pred
        preds /= len(self.models)
        return preds

In [43]:
ensemble1 = Ensemble(path_to_models="weights/final/fold1")
ensemble2 = Ensemble(path_to_models="weights/final/fold2")
ensemble3 = Ensemble(path_to_models="weights/final/fold3")

weights/final/fold1/Egor_resnet-34_fold1__epochepoch=24-val_loss=0.662.ckpt


/home/jovyan/.local/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


weights/final/fold1/Egor_resnet-34_fold1__epochepoch=21-val_loss=0.669.ckpt
weights/final/fold1/Egor_resnet-34_fold1__epochepoch=31-val_loss=0.665.ckpt
weights/final/fold2/Egor_resnet-34_fold2__epochepoch=24-val_loss=0.648.ckpt
weights/final/fold2/Egor_resnet-34_fold2__epochepoch=14-val_loss=0.660.ckpt
weights/final/fold2/Egor_resnet-34_fold2__epochepoch=11-val_loss=0.667.ckpt
weights/final/fold3/Egor_resnet-34_fold3__epochepoch=39-val_loss=0.585.ckpt
weights/final/fold3/Egor_resnet-34_fold3__epochepoch=48-val_loss=0.583.ckpt
weights/final/fold3/Egor_resnet-34_fold3__epochepoch=30-val_loss=0.557.ckpt


In [48]:
from tqdm import tqdm
submission = []
preds1 = []
preds2 = []
preds3 = []

for batch in tqdm(test_loader):
    per_obj_list = []
    per_obj_list.append(int(batch["object_id"][0].item()))

    pred1 = ensemble1.predict(batch)
    preds1.append(pred1)
    
    pred2 = ensemble2.predict(batch)
    preds2.append(pred2)
    
    pred3 = ensemble3.predict(batch)
    preds3.append(pred3)
    
    # pred = 0.3 * pred1 + 0.3 * pred2 + 0.4 * pred3
    pred = pred3

    per_obj_list.append(TARGETS[np.argmax(pred)])
    per_obj_list.append(batch["img_name"][0])
    submission.append(per_obj_list)


 16%|█▌        | 1604/9989 [01:27<07:18, 19.13it/s]Corrupt JPEG data: 42 extraneous bytes before marker 0xd9
Corrupt JPEG data: 36 extraneous bytes before marker 0xd9
 46%|████▌     | 4565/9989 [04:02<04:51, 18.63it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 49%|████▉     | 4941/9989 [04:23<04:43, 17.82it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 50%|████▉     | 4957/9989 [04:24<04:45, 17.64it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
100%|██████████| 9989/9989 [08:58<00:00, 18.55it/s]


In [45]:
s = pd.DataFrame(submission, columns=["object_id", "group", "img_name"])

In [46]:
s

,object_id,group,img_name
0,10239531,Археология,7273147.jpg
1,10239532,Археология,7273149.jpg
2,10314634,Археология,41197649.jpg
3,10314765,Археология,41218675.jpg
4,10332443,Археология,41222250.jpg
...,...,...,...
9984,48002959,"Фото, негативы",59485929.jpg
9985,48509227,"Фото, негативы",60173757.jpg
9986,48509237,"Фото, негативы",60173762.jpg
9987,49040869,"Фото, негативы",61029938.jpg


In [47]:
s.to_csv("submission.csv", sep = ';', encoding = 'utf-8', index = False)